### Initialization

In [1]:
from dash import Dash, html, Output, Input, callback, State, ALL, dash_table
from dash.exceptions import PreventUpdate
import dash.dcc as dcc
import dash_bootstrap_components as dbc
import ta

import yfinance as yf
import os
import pandas as pd
import numpy as np

In [2]:
def grab_ticker_data(symbol, time):
    DATA_PATH = f"../../data/{symbol}_data_{time}.json"
    if os.path.exists(DATA_PATH):
        # Read from file if we've already downloaded the data.
        with open(DATA_PATH) as f:
            ticker_hist = pd.read_json(DATA_PATH)

    else:
        ticker = yf.Ticker(symbol)
        ticker_hist = ticker.history(period=time)

        # Save file to json in case we need it later.  This prevents us from having to re-download it every time.
        ticker_hist.to_json(DATA_PATH)
    return ticker_hist

In [3]:
def create_shifted_data(ticker_hist, days=1):
    # Ensure we know the actual closing price
    data = ticker_hist.copy()
    del data["Stock Splits"]
    del data["Dividends"]
    # data = data.rename(columns = {'Close':'Actual_Close'})
    
    # Setup our target.  This identifies if the price went up or down
    # data["Target"] = ticker_hist.rolling(days+1).apply(lambda x: x.iloc[-1] > x.iloc[0])["Close"] # Might have to adjust if days are different
    # data["Target"] = (ticker_hist["Close"].shift(days) < ticker_hist["Close"]).astype(float)
    data["Target"] = (data["Close"].shift(-days) > data["Close"]).astype(int)

    # Shift stock prices forward one day, so we're predicting tomorrow's stock prices from today's prices.
    # ticker_prev = ticker_hist.copy()
    # ticker_prev = ticker_prev.shift(days)

    # data = data.join(ticker_prev[predictors]).iloc[1:]
    return data

### TA Generator | Dash Preview

In [4]:
symbol = "MSFT"
time = "max"
ticker_hist = grab_ticker_data(symbol, time)
data = create_shifted_data(ticker_hist)

In [5]:
submodules = [name for name in dir(ta) if not (name.startswith("__") or name.startswith("add") or name == 'wrapper' or name == 'utils')]

In [6]:
mod_to_inds = {}
    
for module in submodules:
    function_names = [
        name for name in dir(getattr(ta, module)) 
        if callable(getattr(getattr(ta, module), name)) and 
        not (name.startswith("__") or name.startswith("_") or name[0].isupper())]
    mod_to_inds[module] = function_names

In [52]:
import inspect

def get_function_arguments(func):
    signature = inspect.signature(func)
    arguments = []
    for name, param in signature.parameters.items():
        if param.default is inspect.Parameter.empty:
            arguments.append((name, None))
        else:
            arguments.append((name, param.default))
    return arguments

# list(filter((lambda x: x[0].lower() not in ["close", "open", "high", "low", "volume"]), get_function_arguments(ta.trend.ema_indicator)))

In [55]:
def draw_ind_inputs(module, ind):
    inputs = []
    for arg, val in list(filter(lambda x: (x[0].lower() not in ["close", "open", "high", "low", "volume"]), get_function_arguments(getattr(getattr(ta, module), ind)))):
        inputs.append(dbc.Stack([
            dbc.Label(arg),
            dbc.Input(id={'type':'ind-param', 'index':arg}, value=val)
        ], direction='horizontal', gap=2))
    return inputs


In [56]:
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])
app.layout = dbc.Container([
    dcc.Store(id='ind-store', storage_type='memory'),
    dbc.Row(
        [
            dbc.Col(
                [
                    dbc.Stack([
                        dbc.Label("Indicator Type", style={'width': '20%'}),
                        dcc.Dropdown(submodules, id="submod-dd", style={'width':'100%'}),
                    ], direction='horizontal', gap=2)
                ]
            ),
            dbc.Col(
                [
                    dbc.Stack([
                        dbc.Label("Indicator"),
                        dcc.Dropdown(id = 'ind-dd', style={'width':'100%'})
                    ], direction='horizontal', gap=2)
                ]
            )
        ]
    ),
    html.Br(),
    dbc.Row(
        [
            dbc.Col((
                [
                    html.Div(id='ind-params'),
                    html.Br(),
                    dbc.Button("Add Indicator", id='add-ind', color='primary'),
                    html.Div(id='test-data')
                ]
            ))
        ]
    ),
    dbc.Row(
        [
            dbc.Col([
                dash_table.DataTable(id='ind-table', row_deletable=True, style_cell={'textAlign': 'left'},)
            ])
        ]
    )
])

@app.callback(
    Output('ind-dd', 'options'),
    Input('submod-dd', 'value')
)
def update_ind_dropdown(submod):
    if not submod:
        raise PreventUpdate
    return mod_to_inds[submod]

@app.callback(
    Output('add-ind', 'disabled'), 
    Input('ind-dd', 'value'),
    Input('submod-dd', 'value')
)
def disable_button(ind, mod):
    if not ind or not mod:
        return True
    return False

@app.callback(
    Output('ind-params', 'children'),
    Input('ind-dd', 'value'),
    Input('submod-dd', 'value')
)
def update_ind_adder(ind, mod):
    if not ind or not mod:
        return None
    return draw_ind_inputs(mod, ind)

@app.callback(   
    [Output('ind-store', 'data', allow_duplicate=True),
    Output('ind-table', 'data')],
    [
        Input('add-ind', 'n_clicks'),
        State('ind-store', 'data'),
        State({'type': 'ind-param', 'index': ALL}, 'value'),
        State({'type': 'ind-param', 'index': ALL}, 'id'),
        State('ind-dd', 'value')
    ],
    prevent_initial_call='initial_duplicate'
)
def add_indicator(clicks, data, vals, ids, ind):
    if not clicks or clicks == 0:
        raise PreventUpdate

    if data == None:
        data = {}
    
    data[ind] = {
        ids[i]['index']: vals[i] for i in range(len(ids))
    }

    return data, [{'Indicator': [key]} for key in data.keys()]

@app.callback(
    # [
        # Output('test-data', 'children', allow_duplicate=True),
        Output('ind-store', 'data', allow_duplicate=True),
    # ],
    Input('ind-table', 'data'),
    State('ind-table', 'data_previous'),
    State('ind-store', 'data'),
    prevent_initial_call='initial_duplicate'
)
def remove_data(tdata, old, data):
    if old is None:
        old = []
    else:
        old = list(map(lambda x: x['Indicator'][0], old))
    if tdata is None:
        tdata = []
    else:
        tdata = list(map(lambda x: x['Indicator'][0], tdata))
    
    if len(old) <= len(tdata):
        raise PreventUpdate
    
    for ind in old:
        if ind not in tdata:
            del data[ind]
    return data

@app.callback(
    Output('test-data', 'children'),
    Input('ind-store', 'data')
)
def update_test_data(data):
    if not data: data = {}
    return data.__str__()

app.run(jupyter_mode="inline")